# Description:
To be done

### Imports

In [1]:
#import argparse
#from array import array
import glob
import math
import numpy as np
import os
import ROOT
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, TH3F, TH1, TLine
#from ROOT import gROOT, gBenchmark, gRandom, gSystem, Double, gStyle
import time

Welcome to JupyROOT 6.22/02


### ROOT Batch Mode

In [2]:
# ROOT.gROOT.SetBatch(1)

### Paths

In [3]:
# ============================================================
# Set paths
# ============================================================
datapath = '/data/SingleModule_Nov2020/LArPix/dataRuns/rootTrees/combined_with_light'
print(' datapath:   ', datapath)

outputpath = '/home/lhep/PACMAN/larpix-analysis/lightCharge_anticorrelation'
print(' outputpath: ', outputpath)

files = sorted([os.path.basename(path) for path in glob.glob(datapath+'/*.root')])
print(' datafiles (', len(files), '): ')
for f in files:
    print('              ', f)

 datapath:    /data/SingleModule_Nov2020/LArPix/dataRuns/rootTrees/combined_with_light
 outputpath:  /home/lhep/PACMAN/larpix-analysis/lightCharge_anticorrelation
 datafiles ( 22 ): 
               sync_wLightReco_20201129_122204.root
               sync_wLightReco_20201129_122655.root
               sync_wLightReco_20201129_130449.root
               sync_wLightReco_20201129_134857.root
               sync_wLightReco_20201129_143427.root
               sync_wLightReco_20201129_150749.root
               sync_wLightReco_20201129_155227.root
               sync_wLightReco_20201129_162434.root
               sync_wLightReco_20201129_170918.root
               sync_wLightReco_20201129_174140.root
               sync_wLightReco_20201129_182214.root
               sync_wLightReco_20201129_185437.root
               sync_wLightReco_20201129_193550.root
               sync_wLightReco_20201129_200721.root
               sync_wLightReco_20201129_204607.root
               sync_wLightReco_202011

### Define voxelisation

In [4]:
# ============================================================
# Define voxelisation
# ============================================================
n_pixels_x = 70
n_pixels_y = 70
n_pixels_z = 70
pitch_x = 4.434
pitch_y = 4.434
pitch_z = 4.434
pixels_x_min = - pitch_x * n_pixels_x/2. #155.19
pixels_x_max =   pitch_x * n_pixels_x/2. #155.19
pixels_y_min = - pitch_y * n_pixels_y/2. #155.19
pixels_y_max =   pitch_y * n_pixels_y/2. #155.19
pixels_z_min = 0
pixels_z_max = n_pixels_z*pitch_z*10 # factor 10 for correction... TODO: CHANGE THIS FOR NEXT PRODUCTION!!
#pixels_z_min = - pitch_z * n_pixels_z/2. #155.19
#pixels_z_max =   pitch_z * n_pixels_z/2. #155.19
#pixels_y_min = -155.19
#pixels_y_max = 155.19

### Make TChain with input files

In [5]:
'''
#inputFileName = files[0]
inputFileName = 'sync_wLightReco_20201129_122204.root'
print(' Processing file', inputFileName)

input_tree = ROOT.TChain("t_out", "t_out")
input_tree.Add(datapath + '/' + inputFileName)
#for root_file in config["data_files"]:
#    input_tree.Add(root_file)
#input_tree.Add( "/path/to.root" )

# Turn on all branches of input tree
input_tree.SetBranchStatus("*",           1)
'''
print('Done.')

Done.


### Make List of TChains (each TChain for a different electric field strength)

In [6]:
#inputFileName = files[20]
#inputFileName = 'sync_wLightReco_20201129_122204.root'
#print(' Processing file', inputFileName)

input_tree_list = []

n_field_intensities = 10 # (0.1, 0.2, ..., 0.9, 1.0) kV/cm

for i in range(n_field_intensities):
    input_tree = ROOT.TChain("t_out", "t_out")
    
    if i == 0: # 0.1 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201130_002722.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201130_005847.root')
    if i == 1: # 0.2 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_230700.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_234000.root')
    if i == 2: # 0.3 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_215550.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_222751.root')
    if i == 3: # 0.4 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_204607.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_211819.root')
    if i == 4: # 0.5 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_193550.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_200721.root')
    if i == 5: # 0.6 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_182214.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_185437.root')
    if i == 6: # 0.7 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_170918.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_174140.root')
    if i == 7: # 0.8 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_155227.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_162434.root')
    if i == 8: # 0.9 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_143427.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_150749.root')
    if i == 9: # 1.0 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_122204.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_122655.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_130449.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_134857.root')
    
    # Turn on all branches of input tree
    input_tree.SetBranchStatus("*",           1)
    
    # Append input_tree TChain to the input_tree_list
    input_tree_list.append(input_tree)
    
print(' List with input trees: ')
for i in range(10):
    print('   ', (i+1)/10., 'kV/cm: \t [', input_tree_list[i], '] \t (', input_tree_list[i].GetEntries(), 'entries )')
    
print('Done.')

 List with input trees: 
    0.1 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 4032 entries )
    0.2 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 8914 entries )
    0.3 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 25631 entries )
    0.4 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 31678 entries )
    0.5 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 36235 entries )
    0.6 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 36922 entries )
    0.7 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 37363 entries )
    0.8 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 38061 entries )
    0.9 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 39026 entries )
    1.0 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 60331 entries )
Done.


### Create directories for plots

In [7]:
'''
# If make_plots: Create corresponding folder
make_plots = True

if make_plots:
    plot_folder = inputFileName[16:-5]
    os.system('rm -rf plots/' + str(plot_folder))
    os.system('mkdir plots/' + str(plot_folder))
    os.system('mkdir plots/' + str(plot_folder) + '/eventDisplays')
'''
print('Done.')

Done.


In [8]:
for i in range(len(input_tree_list)):
    plot_folder = str((i+1)/10.) + '_kV_per_cm'
    os.system('rm -rf plots/' + str(plot_folder))
    os.system('mkdir plots/' + str(plot_folder))
    os.system('mkdir plots/' + str(plot_folder) + '/eventDisplays')

### Define track selection cuts

In [9]:
# Define Selection Cuts
cut_ntracks      = 1    # Only consider events with number of reconstructed tracks == cut_ntracks
cut_lenght       = 250. # Track minimum length [mm]
cut_throughgoing = 130. # |max(track_start_y,track_end_y)| and |min(track_start_y,track_end_y)| must be greater than cut_throughgoing [mm]
#cut_nhits        = 50.  # Min. number of hits in the track [-]
#cut_residual2    = 3.   # Maximum allowed sqrt(res_x**2+res_y**2+res_z**2), in order to only allow tracks which are 'track-like' with a small spread

### Loop over all TChains in input_tree_list

In [10]:
track_length_list      = []
track_n_hits_list      = []
track_q_tot_list       = []
track_start_pos_x_list = []
track_start_pos_y_list = []
track_start_pos_z_list = []
track_end_pos_x_list   = []
track_end_pos_y_list   = []
track_end_pos_z_list   = []
LCM_lightYield_list    = []
track_cgx_list         = []
track_cgy_list         = []
track_cgz_list         = []

In [11]:
for chain in range(len(input_tree_list)):
    
    print( ' ----------------------------------------------------------------------------------- ')
    print(' Processing chain Nr.', chain, '(', (chain+1)/10., 'kV/cm )')
    
    #if chain != 0:
    #    continue
    
    # Define lists with interesting quantities
    track_length      = [] # Reconstructed track's length [mm]
    track_n_hits      = [] # Reconstructed track's total number of hits [-]
    track_q_tot       = [] # Reconstructed track's total charge deposit [ke] NOT YET ELECTRON LIFETIME CORRECTED -> TODO)
    track_start_pos_x = [] # Reconstructed track's start position coordinates x [mm]
    track_start_pos_y = [] # Reconstructed track's start position coordinates y [mm]
    track_start_pos_z = [] # Reconstructed track's start position coordinates z [mm]
    track_end_pos_x   = [] # Reconstructed track's end position coordinates x [mm]
    track_end_pos_y   = [] # Reconstructed track's end position coordinates y [mm]
    track_end_pos_z   = [] # Reconstructed track's end position coordinates z [mm]
    LCM_lightYield    = [] # Total reconstructed light yield of LCM
    track_cgx         = [] # gravicentre of the track (weighted by hits_q, but NOT YET ELECTRON LIFETIME CORRECTED -> TODO)
    track_cgy         = [] # gravicentre of the track (weighted by hits_q, but NOT YET ELECTRON LIFETIME CORRECTED -> TODO)
    track_cgz         = [] # gravicentre of the track (weighted by hits_q, but NOT YET ELECTRON LIFETIME CORRECTED -> TODO)
    
    # Number of tracks in the TChain
    n_tracks = input_tree_list[chain].GetEntries()
    print(' Number of tracks : ', n_tracks)

    # Loop over all tracks in input_tree
    start = time.time()

    #print(input_tree_list[chain].Print())

    # List for events with a low light yield
    #low_light_events = []

    # List for track selection
    selected_trackIDs = []
    
    for track_id in range(n_tracks):
        input_tree_list[chain].GetEntry(track_id)

        # Apply Selection Cuts
        if input_tree_list[chain].t_track_length < cut_lenght:
            continue
        if input_tree_list[chain].t_event_ntracks > cut_ntracks:
            continue
        if abs(max(input_tree_list[chain].t_track_start_pos_y,input_tree_list[chain].t_track_end_pos_y)) < cut_throughgoing or\
           abs(min(input_tree_list[chain].t_track_start_pos_y,input_tree_list[chain].t_track_end_pos_y)) < cut_throughgoing:
            #print(input_tree_list[chain].t_track_start_pos_y)
            #print(input_tree_list[chain].t_track_end_pos_y)
            #print(' ---------------- ')
            continue

        #if track_id > 200:
        #    break
        
        selected_trackIDs.append(input_tree_list[chain].t_trackID)


        #if track_id > 15:
        #    break

        '''        
        print(' ----------------------------------------- ')
        #print(' t_eventID:           ', input_tree_list[chain].t_eventID)
        print(' t_trackID:           ', input_tree_list[chain].t_trackID)
        #print(' t_event_q:           ', input_tree_list[chain].t_event_q)
        print(' t_track_length:      ', input_tree_list[chain].t_track_length)
        print(' t_event_ntracks:     ', input_tree_list[chain].t_event_ntracks)
        print(' t_track_start_pos_y: ', input_tree_list[chain].t_track_start_pos_y)
        print(' t_track_end_pos_y:   ', input_tree_list[chain].t_track_end_pos_y)
        #print(' t_track_q:           ', input_tree_list[chain].t_track_q)
        #print(' t_event_nhits:       ', input_tree_list[chain].t_event_nhits)
        #print(' t_track_nhits:       ', input_tree_list[chain].t_track_nhits)
        #print(' t_tlcm1_tot:         ', input_tree_list[chain].l_tlcm1_tot)
        '''

        track_length.append(input_tree_list[chain].t_track_length)
        track_n_hits.append(input_tree_list[chain].t_track_nhits)
        track_q_tot.append(input_tree_list[chain].t_track_q)

        track_start_pos_x.append(input_tree_list[chain].t_track_start_pos_x)
        track_start_pos_y.append(input_tree_list[chain].t_track_start_pos_y)
        track_start_pos_z.append(input_tree_list[chain].t_track_start_pos_z)
        track_end_pos_x.append(input_tree_list[chain].t_track_end_pos_x)
        track_end_pos_y.append(input_tree_list[chain].t_track_end_pos_y)
        track_end_pos_z.append(input_tree_list[chain].t_track_end_pos_z)

        LCM_lightYield.append(input_tree_list[chain].l_tlcm1_tot)

        # For low light events
        #if input_tree_list[chain].l_tlcm1_tot<150:
        #    low_light_events.append(input_tree_list[chain].t_trackID)

        # Obtain center of gravity for the track (weighted by deposited charge) #TODO: Correct for the electron lifetime!!!
        # Loop over all hits
        cgx = 0.
        cgy = 0.
        cgz = 0.
        for hit in range(input_tree_list[chain].t_track_nhits):
            cgx += input_tree_list[chain].t_track_hits_x[hit]*input_tree_list[chain].t_track_hits_q[hit] # TODO: CORRECT FOR ELECTRON LIFETIME
            cgy += input_tree_list[chain].t_track_hits_y[hit]*input_tree_list[chain].t_track_hits_q[hit] # TODO: CORRECT FOR ELECTRON LIFETIME
            cgz += input_tree_list[chain].t_track_hits_z[hit]*input_tree_list[chain].t_track_hits_q[hit] # TODO: CORRECT FOR ELECTRON LIFETIME
        # normalize...
        gravicentre_x = cgx/(np.sum(input_tree_list[chain].t_track_hits_q))
        gravicentre_y = cgy/(np.sum(input_tree_list[chain].t_track_hits_q))
        gravicentre_z = cgz/(np.sum(input_tree_list[chain].t_track_hits_q))
        track_cgx.append(gravicentre_x)
        track_cgy.append(gravicentre_y)
        track_cgz.append(gravicentre_z)
        #print(' ------------------------------ ')
        #print(' t_trackID:     ', input_tree_list[chain].t_trackID)
        #print(' cgx:           ', gravicentre_x)
        #print(' cgy:           ', gravicentre_y)
        #print(' cgz:           ', gravicentre_z)

        if(track_id%200==0):
            now = time.time()
            print(' Processed', track_id+1, 'of', n_tracks, 'tracks. \t Elapsed time:', (now-start), ' seconds ... \r')
    
    track_length_list.append(track_length)
    track_n_hits_list.append(track_n_hits)
    track_q_tot_list.append(track_q_tot)
    track_start_pos_x_list.append(track_start_pos_x)
    track_start_pos_y_list.append(track_start_pos_y)
    track_start_pos_z_list.append(track_start_pos_z)
    track_end_pos_x_list.append(track_end_pos_x)
    track_end_pos_y_list.append(track_end_pos_y)
    track_end_pos_z_list.append(track_end_pos_z)
    LCM_lightYield_list.append(LCM_lightYield)
    track_cgx_list.append(track_cgx)
    track_cgy_list.append(track_cgy)
    track_cgz_list.append(track_cgz)
            
    print(' Number of selected tracks: ', len(selected_trackIDs))
    
print(' Done.')

 ----------------------------------------------------------------------------------- 
 Processing chain Nr. 0 ( 0.1 kV/cm )
 Number of tracks :  4032
 Number of selected tracks:  6
 ----------------------------------------------------------------------------------- 
 Processing chain Nr. 1 ( 0.2 kV/cm )
 Number of tracks :  8914
 Processed 2001 of 8914 tracks. 	 Elapsed time: 0.23361921310424805  seconds ... 
 Number of selected tracks:  123
 ----------------------------------------------------------------------------------- 
 Processing chain Nr. 2 ( 0.3 kV/cm )
 Number of tracks :  25631
 Processed 3601 of 25631 tracks. 	 Elapsed time: 0.7914800643920898  seconds ... 
 Processed 6401 of 25631 tracks. 	 Elapsed time: 1.4550752639770508  seconds ... 
 Processed 11401 of 25631 tracks. 	 Elapsed time: 2.633225440979004  seconds ... 
 Processed 12001 of 25631 tracks. 	 Elapsed time: 2.768171548843384  seconds ... 
 Processed 13801 of 25631 tracks. 	 Elapsed time: 3.1677768230438232  secon

### Make Plots for Selected (!) Events

In [12]:
from plot_functions import *

In [13]:
# LCM_lightYield
for chain in range(len(input_tree_list)):
    #print(LCM_lightYield_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min  = np.min(LCM_lightYield_list[chain])
    x_max  = np.max(LCM_lightYield_list[chain])
    n_bins = 50
    axis_labels = ['Reconstructed LCM Light Yield [-]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h1_LCM_lightYield.png'
    plot_h1(LCM_lightYield_list[chain],x_min,x_max,n_bins,axis_labels,save_name)
print('Done.')

Done.


In [14]:
# Track Length
for chain in range(len(input_tree_list)):
    #print(track_length_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min  = 0
    x_max  = np.max(track_length_list[chain])
    n_bins = 50
    axis_labels = ['Track Length [mm]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h1_track_lengths.png'
    plot_h1(track_length_list[chain],x_min,x_max,n_bins,axis_labels,save_name)
print('Done.')

Done.


In [15]:
# Track n_hits
for chain in range(len(input_tree_list)):
    #print(track_n_hits_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min  = 0
    x_max  = np.max(track_n_hits_list[chain])
    n_bins = 50
    axis_labels = ['Number of Hits [-]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h1_track_nhits.png'
    plot_h1(track_n_hits_list[chain],x_min,x_max,n_bins,axis_labels,save_name)
print('Done.')

Done.


In [16]:
# Track q_tot
for chain in range(len(input_tree_list)):
    #print(track_q_tot_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min  = 0
    x_max  = np.max(track_q_tot_list[chain])
    n_bins = 50
    axis_labels = ['Total Deposited Charge [ke]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h1_track_qtot.png'
    plot_h1(track_q_tot_list[chain],x_min,x_max,n_bins,axis_labels,save_name)
print('Done.')

Done.


In [17]:
# start_pos_y vs. start_pos_x
for chain in range(len(input_tree_list)):
    #print(track_start_pos_x_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min    = pixels_x_min # np.min(track_start_pos_x_list[chain])
    x_max    = pixels_x_max # np.max(track_start_pos_x_list[chain])
    y_min    = pixels_y_min # np.min(track_start_pos_y_list[chain])
    y_max    = pixels_y_max # np.max(track_start_pos_y_list[chain])
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    #x_bins = np.arange(x_min,x_max+1,step=1.)
    #y_bins = np.arange(y_min,y_max+1,step=1.)
    axis_labels = ['Track Start Position x [mm]','Track Start Position y [mm]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h2_track_startPos.png'
    plot_h2([track_start_pos_x_list[chain],track_start_pos_y_list[chain]],x_bins,y_bins,axis_labels,save_name)
print('Done.')

/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 8722 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:201: RuntimeWarning: Glyph 8722 missing from current font.
  font.set_text(s, 0, flags=flags)


Done.


In [18]:
# end_pos_y vs. end_pos_x
for chain in range(len(input_tree_list)):
    #print(track_end_pos_x_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min    = pixels_x_min # np.min(track_start_pos_x_list[chain])
    x_max    = pixels_x_max # np.max(track_start_pos_x_list[chain])
    y_min    = pixels_y_min # np.min(track_start_pos_y_list[chain])
    y_max    = pixels_y_max # np.max(track_start_pos_y_list[chain])
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    #x_bins = np.arange(x_min,x_max+1,step=1.)
    #y_bins = np.arange(y_min,y_max+1,step=1.)
    axis_labels = ['Track End Position x [mm]','Track End Position y [mm]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h2_track_endPos.png'
    plot_h2([track_end_pos_x_list[chain],track_end_pos_y_list[chain]],x_bins,y_bins,axis_labels,save_name)
print('Done.')

Done.


In [19]:
# NHits vs. TrackLength
for chain in range(len(input_tree_list)):
    #print(track_length_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min    = 0
    x_max    = 500
    y_min    = 0
    y_max    = 200
    n_bins_x = 50
    n_bins_y = 50
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    #x_bins = np.arange(x_min,x_max+1,step=1.)
    #y_bins = np.arange(y_min,y_max+1,step=1.)
    axis_labels = ['Track Length [mm]','Number of Hits [-]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h2_trackLength_vs_trackNHits.png'
    plot_h2([track_length_list[chain],track_n_hits_list[chain]],x_bins,y_bins,axis_labels,save_name)
print('Done.')

Done.


In [20]:
# 3D plot of gravicentres
for chain in range(len(input_tree_list)):
    #print(track_cgx_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    y_min    = pixels_y_min
    y_max    = pixels_y_max
    z_min    = 0
    z_max    = max(pixels_z_max,np.max(track_cgz_list[chain]))
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    n_bins_z = n_pixels_z
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    z_bins = np.linspace(z_min,z_max,n_bins_z)
    save_name = 'plots/'+str(plot_folder)+'/'+'h3_graviCentre.png'
    plot_h3_unweighted([track_cgx_list[chain],track_cgy_list[chain],track_cgz_list[chain]],x_bins,y_bins,z_bins,['Coordinate x [mm]','Coordinate y [mm]','Coordinate z [mm]'],save_name)

In [21]:
# Gravycentre projection onto xy plane
for chain in range(len(input_tree_list)):
    #print(track_cgx_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    y_min    = pixels_y_min
    y_max    = pixels_y_max
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    #x_bins = np.arange(x_min,x_max+1,step=1.)
    #y_bins = np.arange(y_min,y_max+1,step=1.)
    axis_labels = ['Coordinate x [mm]','Coordinate y [mm]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h2_graviCentre_xy.png'
    plot_h2([track_cgx_list[chain],track_cgy_list[chain]],x_bins,y_bins,axis_labels,save_name)
print('Done.')

Done.


In [22]:
# Gravycentre projection onto xz plane
for chain in range(len(input_tree_list)):
    #print(track_cgx_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    z_min    = 0
    z_max    = max(pixels_z_max,np.max(track_cgz_list[chain]))
    n_bins_x = n_pixels_x
    n_bins_z = n_pixels_z
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    z_bins = np.linspace(z_min,z_max,n_bins_z)
    #x_bins = np.arange(x_min,x_max+1,step=1.)
    #y_bins = np.arange(y_min,y_max+1,step=1.)
    axis_labels = ['Coordinate x [mm]','Coordinate z [mm]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h2_graviCentre_xz.png'
    plot_h2([track_cgx_list[chain],track_cgz_list[chain]],x_bins,z_bins,axis_labels,save_name)
print('Done.')

Done.


In [23]:
# Gravycentre projection onto yz plane
for chain in range(len(input_tree_list)):
    #print(track_cgy_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    y_min    = pixels_y_min
    y_max    = pixels_y_max
    z_min    = 0
    z_max    = max(pixels_z_max,np.max(track_cgz_list[chain]))
    n_bins_y = n_pixels_y
    n_bins_z = n_pixels_z
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    z_bins = np.linspace(z_min,z_max,n_bins_z)
    #y_bins = np.arange(y_min,y_max+1,step=1.)
    #z_bins = np.arange(z_min,z_max+1,step=1.)
    axis_labels = ['Coordinate y [mm]','Coordinate z [mm]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h2_graviCentre_yz.png'
    plot_h2([track_cgy_list[chain],track_cgz_list[chain]],y_bins,z_bins,axis_labels,save_name)
print('Done.')

Done.


In [24]:
# LCM light-yield vs. track gravicentre_x
for chain in range(len(input_tree_list)):
    #print(track_cgx_list[chain])
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    y_min    = 0 # np.min(LCM_lightYield_list[chain])
    y_max    = 1500 # np.max(LCM_lightYield_list[chain])
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    axis_labels = ['Track Gravicentre x [mm]','LCM Light Yield [UNITS?]','Entries [-]']
    save_name = 'plots/'+str(plot_folder)+'/'+'h2_LCM_lightYield_vs_trackGC.png'
    plot_h2([track_cgx_list[chain],LCM_lightYield_list[chain]],x_bins,y_bins,axis_labels,save_name)
print('Done.')

Done.


In [25]:
# LCM light-yield vs. track gravicentre_x, all with selection cut line, and only selected with selection cut line
x_min    = pixels_x_min
x_max    = pixels_x_max
y_min    = 0 # np.min(LCM_lightYield_list[chain])
y_max    = 1500 # np.max(LCM_lightYield_list[chain])
n_bins_x = n_pixels_x
n_bins_y = n_pixels_y
x_bins = np.linspace(x_min,x_max,n_bins_x)
y_bins = np.linspace(y_min,y_max,n_bins_y)
axis_labels = ['Track Gravicentre x [mm]','LCM Light Yield [UNITS?]','Entries [-]']

LY_cut_at_xmin = 400 # min. light yield at x_min (near end)
LY_cut_at_xmax = 50  # min. light yield at x_max (far end)

# Plot only selected
for chain in range(len(input_tree_list)):
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    save_name = 'plots/'+str(plot_folder)+'/h2_LCM_lightYield_vs_trackGC_selected.png'
    gc_list = []
    ly_list = []
    for entry in range(len(LCM_lightYield_list[chain])):
        if LCM_lightYield_list[chain][entry]>-(track_cgx_list[chain][entry]-x_min)*(LY_cut_at_xmin-LY_cut_at_xmax)/(x_max-x_min)+LY_cut_at_xmin:
            gc_list.append(track_cgx_list[chain][entry])
            ly_list.append(LCM_lightYield_list[chain][entry])
    plot_LY_vs_GCx_selection([gc_list,ly_list],x_bins,y_bins,axis_labels,save_name,LY_cut_at_xmin,LY_cut_at_xmax)
    
# Plot all
for chain in range(len(input_tree_list)):
    plot_folder = str((chain+1)/10.) + '_kV_per_cm'
    save_name = 'plots/'+str(plot_folder)+'/h2_LCM_lightYield_vs_trackGC_selection_cut.png'
    gc_list = []
    ly_list = []
    for entry in range(len(LCM_lightYield_list[chain])):
        if True:
            gc_list.append(track_cgx_list[chain][entry])
            ly_list.append(LCM_lightYield_list[chain][entry])
    plot_LY_vs_GCx_selection([gc_list,ly_list],x_bins,y_bins,axis_labels,save_name,LY_cut_at_xmin,LY_cut_at_xmax)

print('Done.')

Done.


In [26]:
# Profile plot of LCM light-yield vs. track gravicentre_x
x_min    = pixels_x_min
x_max    = pixels_x_max
n_bins_x = 10
x_width  = (x_max-x_min)/n_bins_x
x_err    = [0.5*x_width for i in range(n_bins_x)]
y_min    = 0 # np.min(LCM_lightYield_list[chain])
y_max    = 1500 # np.max(LCM_lightYield_list[chain])

# Define lists which will be filled with n (=number of TChains) lists with info about
# gravicentre, mean light yield and standard deviation of the mean light yield
gc_bin_i_list      = []
ly_mean_bin_i_list = []
ly_std_bin_i_list  = []
plot_folder_list   = []

# Loop over all TChains and obtain data for the profile plots
for chain in range(len(input_tree_list)):
    #print(track_cgx_list[chain])
    gc_bin_i      = []
    ly_mean_bin_i = []
    ly_std_bin_i  = []
    # Loop over all bins
    for bin_i in range(n_bins_x):
        gc_bin_i.append((bin_i+0.5)*x_width+x_min)
        ly_in_bin_i = []
        # Loop over all TChains
        for entry in range(len(LCM_lightYield_list[chain])):
            # Check if GC is in current bin
            if track_cgx_list[chain][entry]>((bin_i)*x_width+x_min) and track_cgx_list[chain][entry]<((bin_i+1)*x_width+x_min):
                # Check if LY is not below selection criterium
                if LCM_lightYield_list[chain][entry]>-(track_cgx_list[chain][entry]-x_min)*(LY_cut_at_xmin-LY_cut_at_xmax)/(x_max-x_min)+LY_cut_at_xmin:
                    ly_in_bin_i.append(LCM_lightYield_list[chain][entry])
        ly_mean_bin_i.append(np.mean(ly_in_bin_i))
        ly_std_bin_i.append(np.std(ly_in_bin_i))
    gc_bin_i_list.append(gc_bin_i)
    ly_mean_bin_i_list.append(ly_mean_bin_i)
    ly_std_bin_i_list.append(ly_std_bin_i)
    plot_folder_list.append(str((chain+1)/10.) + '_kV_per_cm')

axis_labels = ['Track Gravicentre x [mm]','LCM Light Yield [UNITS?]']
save_name = 'plots/profile2D_LCM_lightYield_vs_trackGC.png'
plot_profile_of_2D_hist(gc_bin_i_list,\
                        ly_mean_bin_i_list,\
                        x_err,\
                        ly_std_bin_i_list,\
                        plot_folder_list,\
                        axis_labels,\
                        x_min,\
                        x_max,\
                        y_min,\
                        y_max,\
                        save_name)
print('Done.')

/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
 

Done.


#### Event Display

In [29]:
#os.system('rm -rf plots/' + str(plot_folder) +'/eventDisplays')
#os.system('mkdir plots/' + str(plot_folder) +'/eventDisplays')

# Define which TChain to be used
chain = 0

input_tree = input_tree_list[chain]
n_tracks = input_tree.GetEntries()

print(' n_tracks:          ', input_tree.GetEntries())

for track in range(n_tracks):
    input_tree.GetEntry(track)
    
    if track%200==0:
        print(' Processing track', track, 'of', n_tracks, 'in total...')

    # Apply Selection Cuts (those should be the same as at the very beginning!)
    # Note: Make a selected_trackID list does not work since there are >1 files in a TChain.
    # Could, e.g., use unique unix_ts or so.
    # For the moment, just apply exactly the same selection cuts as above.
    if input_tree_list[chain].t_track_length < cut_lenght:
        continue
    if input_tree_list[chain].t_event_ntracks > cut_ntracks:
        continue
    if abs(max(input_tree_list[chain].t_track_start_pos_y,input_tree_list[chain].t_track_end_pos_y)) < cut_throughgoing or\
       abs(min(input_tree_list[chain].t_track_start_pos_y,input_tree_list[chain].t_track_end_pos_y)) < cut_throughgoing:
        #print(input_tree_list[chain].t_track_start_pos_y)
        #print(input_tree_list[chain].t_track_end_pos_y)
        #print(' ---------------- ')
        continue
    
    hits_x = []
    hits_y = []
    hits_z = []
    hits_q = []

    '''    
    print(' ----------------------------------------- ')
    #print(' t_eventID:        ', input_tree.t_eventID)
    print(' t_trackID:        ', input_tree.t_trackID)
    #print(' t_event_q:        ', input_tree.t_event_q)
    #print(' t_track_q:        ', input_tree.t_track_q)
    #print(' t_event_nhits:    ', input_tree.t_event_nhits)
    print(' t_track_start:    [%.2f , %.2f , %.2f]' %(input_tree.t_track_start_pos_x,input_tree.t_track_start_pos_y,input_tree.t_track_start_pos_z))
    print(' t_track_end:      [%.2f , %.2f , %.2f]' %(input_tree.t_track_end_pos_x,input_tree.t_track_end_pos_y,input_tree.t_track_end_pos_z))
    #print(' max(start_y):     ', max(input_tree.t_track_start_pos_y,input_tree.t_track_end_pos_y))
    #print(' min(start_y):     ', min(input_tree.t_track_start_pos_y,input_tree.t_track_end_pos_y))
    print(' t_track_nhits:    ', input_tree.t_track_nhits)
    #print(' t_track_res_x:    ', input_tree.t_track_residual_x)
    #print(' t_track_res_y:    ', input_tree.t_track_residual_y)
    #print(' t_track_res_z:    ', input_tree.t_track_residual_z)
    #print(' sqrt(res_sum2):   ', np.sqrt(input_tree.t_track_residual_x**2 + input_tree.t_track_residual_y**2 + input_tree.t_track_residual_z**2  ))
    
    #print(' l_LCM_tot :      ', input_tree.l_tlcm1_tot)
    '''
    
    # Loop over all hits
    for hit in range(input_tree.t_track_nhits):
        hits_x.append(input_tree.t_track_hits_x[hit])
        hits_y.append(input_tree.t_track_hits_y[hit])
        hits_z.append(input_tree.t_track_hits_z[hit])
        hits_q.append(input_tree.t_track_hits_q[hit])
    
    
    # Plot 2D projection (onto xy pixel tile)
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    y_min    = pixels_y_min
    y_max    = pixels_y_max
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    #save_name = 'plots/'+str(plot_folder)+'/eventDisplays/trackID_'+str(input_tree.t_trackID)+'_2D.png'
    save_name = 'plots/'+str((chain+1)/10.)+'_kV_per_cm/eventDisplays/trackID_'+str(input_tree.t_trackID)+'_2D.png'
    plot_event_2D([hits_x,hits_y,hits_q],x_bins,y_bins,['Coordinate x [mm]','Coordinate y [mm]','Charge Deposition [ke]'],save_name)
    
    
    # Plot 3D event display
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    y_min    = pixels_y_min
    y_max    = pixels_y_max
    z_min    = 0
    z_max    = max(pixels_z_max,np.max(hits_z))
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    n_bins_z = n_pixels_z
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    z_bins = np.linspace(z_min,z_max,n_bins_z)
    #save_name = 'plots/'+str(plot_folder)+'/eventDisplays/trackID_'+str(input_tree.t_trackID)+'_3D.png'
    save_name = 'plots/'+str((chain+1)/10.)+'_kV_per_cm/eventDisplays/trackID_'+str(input_tree.t_trackID)+'_3D.png'
    plot_event_3D([hits_x,hits_y,hits_z,hits_q],x_bins,y_bins,z_bins,['Coordinate x [mm]','Coordinate y [mm]','Coordinate z [mm]','Charge Deposition [ke]'],save_name)
    '''
    # OLD VERSION
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim((pixels_x_min,pixels_x_max))
    ax.set_ylim((pixels_y_min,pixels_y_max))
    ax.set_zlim((0.,max(3200,np.max(hits_z))))
    img = ax.scatter(hits_x,hits_y,hits_z,c=hits_q,cmap=plt.hot())
    fig.colorbar(img)
    plt.show()
    save_name = 'evDisplay_3D.png'
    plt.savefig(save_name, dpi=400)
    plt.close()
    '''
    
print('Done.')

 n_tracks:           4032
 Processing track 0 of 4032 in total...
 Processing track 200 of 4032 in total...
 Processing track 400 of 4032 in total...
 Processing track 600 of 4032 in total...
 Processing track 800 of 4032 in total...
 Processing track 1000 of 4032 in total...
 Processing track 1200 of 4032 in total...
 Processing track 1400 of 4032 in total...
 Processing track 1600 of 4032 in total...
 Processing track 1800 of 4032 in total...
 Processing track 2000 of 4032 in total...
 Processing track 2200 of 4032 in total...
 Processing track 2400 of 4032 in total...
 Processing track 2600 of 4032 in total...
 Processing track 2800 of 4032 in total...
 Processing track 3000 of 4032 in total...
 Processing track 3200 of 4032 in total...
 Processing track 3400 of 4032 in total...
 Processing track 3600 of 4032 in total...
 Processing track 3800 of 4032 in total...
 Processing track 4000 of 4032 in total...
Done.


### Testing Voxelisation algorithm

In [ ]:
'''
# Get all hits in the EVENT (!) and make voxelisation
voxels = np.zeros((n_voxels_x, n_voxels_y, n_voxels_z))

for hit in range(10): #input_tree.t_event_nhits):
    #print(' hit: ', hit, ' \t x: ', input_tree.t_event_hits_x[hit], '\t y: ', input_tree.t_event_hits_y[hit], ' \t z: ', input_tree.t_event_hits_z[hit])

    voxel_x = math.floor((input_tree.t_event_hits_x[hit]+(pitch_x*(n_voxels_x)/2.))/pitch_x)
    voxel_y = math.floor((input_tree.t_event_hits_y[hit]+(pitch_y*(n_voxels_y)/2.))/pitch_y)
    voxel_z = math.floor((input_tree.t_event_hits_z[hit]+(pitch_z*(n_voxels_z)/2.))/pitch_z)

    #print(' voxel_x: ', voxel_x, ' \t voxel_y: ', voxel_y, ' \t voxel_z: ', voxel_z)
    if voxel_x<n_voxels_x and voxel_y<n_voxels_y and voxel_z<n_voxels_z:
        voxels[voxel_x][voxel_y][voxel_z] += input_tree.t_event_hits_q[hit]
    # TODO: make under- and overflow voxel for every coordinate

    h3_event_hits.Fill(input_tree.t_event_hits_x[hit],input_tree.t_event_hits_y[hit],input_tree.t_event_hits_z[hit],input_tree.t_event_hits_q[hit])

for vox_x in range(n_voxels_x):
    vox_x_middle = x_min + (vox_x+0.5)*pitch_x
    for vox_y in range(n_voxels_y):
        vox_y_middle = y_min + (vox_y+0.5)*pitch_y
        for vox_z in range(n_voxels_z):
            vox_z_middle = z_min + (vox_z+0.5)*pitch_z
            if voxels[vox_x][vox_y][vox_z] > 0:
                ntuple.Fill(vox_x_middle,vox_y_middle,vox_z_middle,voxels[vox_x][vox_y][vox_z])
                #h3_event_hits.Fill(vox_x_middle,vox_y_middle,vox_z_middle,voxels[vox_x][vox_y][vox_z])
'''